# The Keras functional API

## Introduction to the functional API

In [1]:
from keras import Input, layers
from keras.models import Model, Sequential

seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

# Equivalent in the functional API
input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = Model(input_tensor, output_tensor)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [2]:
# Compiling, training and evaluating the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

import numpy as np
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

model.fit(x_train, y_train, epochs=10, batch_size=128)

score = model.evaluate(x_train, y_train)

Epoch 1/10
8/8 [==============================] - 0s 19ms/step - loss: 12.0441
Epoch 2/10
8/8 [==============================] - ETA: 0s - loss: 13.128/8 [==============================] - 0s 50ms/step - loss: 13.1236
Epoch 3/10
8/8 [==============================] - 0s 33ms/step - loss: 15.1772
Epoch 4/10
8/8 [==============================] - 0s 19ms/step - loss: 17.7597
Epoch 5/10
8/8 [==============================] - 0s 24ms/step - loss: 21.4344
Epoch 6/10
8/8 [==============================] - 0s 33ms/step - loss: 26.1615
Epoch 7/10
8/8 [==============================] - 0s 20ms/step - loss: 31.7835
Epoch 8/10
8/8 [==============================] - 0s 19ms/step - loss: 37.9756
Epoch 9/10
8/8 [==============================] - 0s 25ms/step - loss: 44.5488
Epoch 10/10
32/32 [==============================] - 0s 10ms/step - loss: 55.4539


## Multi-input models

In [1]:
# Functional API implementation of a two-input question-answering model
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)
model.summary()
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None, None)]       0                                            
__________________________________________________________________________________________________
question (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 64)     640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 32)     320000      question[0][0]                   
______________________________________________________________________________________________

In [3]:
# Feeding data to a multi-input model
import numpy as np
from keras.utils import to_categorical

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))
answers = np.random.randint(answer_vocabulary_size, size=(num_samples))
answers = to_categorical(answers, answer_vocabulary_size)

# Using a list of inputs
print("Using a list of inputs:")
model.fit([text, question], answers, epochs=10, batch_size=128)
print()
# Using a dict of inputs (only if inputs are named)
print("Using a dict of inputs")
model.fit({'text':text, 'question':question}, answers, epochs=10, batch_size=128)

Using a list of inputs:
Epoch 1/10
8/8 [==============================] - 0s 50ms/step - loss: 6.2148 - acc: 0.0010
Epoch 2/10
8/8 [==============================] - 0s 52ms/step - loss: 6.1988 - acc: 0.0390
Epoch 3/10
8/8 [==============================] - 0s 35ms/step - loss: 6.1630 - acc: 0.0230
Epoch 4/10
8/8 [==============================] - 0s 42ms/step - loss: 6.0730 - acc: 0.0070
Epoch 5/10
8/8 [==============================] - 0s 51ms/step - loss: 5.9971 - acc: 0.0120
Epoch 6/10
8/8 [==============================] - 0s 44ms/step - loss: 5.8894 - acc: 0.0170
Epoch 7/10
8/8 [==============================] - 0s 35ms/step - loss: 5.7925 - acc: 0.0240
Epoch 8/10
8/8 [==============================] - 0s 38ms/step - loss: 5.7065 - acc: 0.0350
Epoch 9/10
8/8 [==============================] - 0s 39ms/step - loss: 5.6293 - acc: 0.0330
Epoch 10/10
8/8 [==============================] - 0s 44ms/step - loss: 5.5621 - acc: 0.0500

Using a dict of inputs
Epoch 1/10
8/8 [===============

## Multi-output models

In [1]:
# Three-output model
from keras import layers
from keras import Input
from keras.models import Model

vocabulary_size = 50000
num_income_groupes = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(vocabulary_size, 256)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groupes, activation='softmax', name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

In [2]:
# Compilation
model.compile(optimizer='rmsprop', loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])

# Equivalent
model.compile(optimizer='rmsprop',
              loss={'age': 'mse',
                    'income': 'categorical_crossentropy',
                    'gender': 'binary_crossentropy'})

In [3]:
# Loss weighting
model.compile(optimizer='rmsprop',
              loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'],
              loss_weights=[0.25, 1., 10.])

In [4]:
# Feeding data to a multi-output model
# model.fit(posts, [age_targets, income_targets, gender_targets], epochs=10, batch_size=64)

## Directed acyclic graphs of layers

### Inception module

In [7]:
# An Inception model
from keras import layers
from keras import Input

x = Input(shape=(28, 28, 1), dtype='float32', name='images')
print("x shape:", x.shape)

branch_a = layers.Conv2D(128, 1, activation='relu', padding='same', strides=2)(x)

branch_b = layers.Conv2D(128, 1, activation='relu', padding='same')(x)
branch_b = layers.Conv2D(128, 3, activation='relu', strides=2, padding='same')(branch_b)

branch_c = layers.AveragePooling2D(3, strides=2, padding='same')(x)
branch_c = layers.Conv2D(128, 3, activation='relu', padding='same')(branch_c)

branch_d = layers.Conv2D(128, 1, activation='relu', padding='same')(x)
branch_d = layers.Conv2D(128, 3, activation='relu', padding='same')(branch_d)
branch_d = layers.Conv2D(128, 3, activation='relu', strides=2, padding='same')(branch_d)

output = layers.concatenate([branch_a, branch_b, branch_c, branch_d], axis=-1)

x shape: (None, 28, 28, 1)


### Residual connections

In [8]:
# feature map sizes are the same
from keras import layers
from keras import Input

x = Input(shape=(28, 28, 1), dtype='float32', name='images')

y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)

y = layers.add([y, x])

In [9]:
# feature map sizes differ
from keras import layers
from keras import Input

x = Input(shape=(28, 28, 1), dtype='float32', name='images')

y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.MaxPooling2D(2, strides=2)(y)

residual = layers.Conv2D(128, 1, strides=2, padding='same')(x)

y = layers.add([y, residual])

## Layer weight sharing

In [10]:
# semantic similarity between 2 sentences using layer sharing
from keras import layers
from keras import Input
from keras.models import Model

lstm = layers.LSTM(32)

left_input = Input(shape=(None, 128))
left_output = lstm(left_input)

right_input = Input(shape=(None, 128))
right_output = lstm(right_input)

merged = layers.concatenate([left_output, right_output], axis=-1)
predictions = layers.Dense(1, activation='sigmoid')(merged)

model = Model([left_input, right_input], predictions)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 128)]  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 128)]  0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 32)           20608       input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 64)           0           lstm[0][0]                 

## Models as layers

In [11]:
# Siamese vision model
from keras import layers
from keras import Input
from keras import applications

xception_base = applications.Xception(weights=None, include_top=False)

left_input = Input(shape=(250, 250, 3))
right_input = Input(shape=(250, 250, 3))

left_features = xception_base(left_input)
right_features = xception_base(right_input)

merged_features = layers.concatenate([left_features, right_features], axis=-1)

predictions = layers.Dense(10, activation='softmax')(merged_features)

model = Model([left_input, right_input], predictions)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 250, 250, 3) 0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 250, 250, 3) 0                                            
__________________________________________________________________________________________________
xception (Model)                multiple             20861480    input_4[0][0]                    
                                                                 input_5[0][0]                    
__________________________________________________________________________________________________
concatenate_3 (Concatenate)     (None, 8, 8, 4096)   0           xception[1][0]             